In [1]:
import pandas as pd
import numpy as np
import random

In [5]:
# QUESTION 1
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P data/
data = pd.read_csv('data/data_000637.txt')
N = (int(1e4) + 1)

data_head = data.head(N)
print(data_head)

--2022-12-10 18:32:06--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2022-12-10 18:32:07--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc12fb8c6d7591279cc3b8969b54.dl.dropboxusercontent.com/cd/0/inline/ByV4NISJu1LObIpaCp7DzF9kc5qcZO1A6GoyKxKxAVI4cM5YW7EcHrrG-10HCJE_chCIo0441kR5Y1s_Jazd0LbhyYuidwN9cGQVFq3udWl9bvQFVcFGlDnUTkOu6-EO2iYHrRfc0YsfKLol1RrKbNIdgIB73-9AIM49DukCHAotuA/file# [following]
--2022-12-10 18:32:07--  https://uc12fb8c6d7591279cc3b8969b54.dl.dropboxusercontent.com/cd/0/inline/ByV4NISJu1LObIpaCp7DzF9kc5qcZO1A6GoyKxKxAVI4cM5YW7EcHrrG-10HCJE_chCIo0441kR5Y1s_Jazd0LbhyYuidwN9c

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0         1     0          123  3869200167        2374        26
1         1     0          124  3869200167        2374        27
2         1     0           63  3869200167        2553        28
3         1     0           64  3869200167        2558        19
4         1     0           64  3869200167        2760        25
...     ...   ...          ...         ...         ...       ...
9996      1     0           60  3869200267         701         5
9997      1     0           59  3869200267         707        23
9998      1     0           63  3869200267         706        15
9999      1     0           49  3869200267         777        13
10000     1     0           50  3869200267         776        22

[10001 rows x 6 columns]


In [4]:
#QUESTION 2
orbit_unit = data_head['BX_COUNTER'].max()
print(orbit_unit)

3563


In [6]:
#QUESTION 3
counter_bx = np.array(data_head['BX_COUNTER'])
counter_orbit = np.array(data_head['ORBIT_CNT'])
time_ns = (counter_bx+counter_orbit*orbit_unit)*25
data_head['TIME_IN_NS'] = time_ns
time_new = pd.to_datetime(time_ns, unit='ns')
data_head['TIME_SRS'] = time_new
print(data_head)

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0         1     0          123  3869200167        2374        26   
1         1     0          124  3869200167        2374        27   
2         1     0           63  3869200167        2553        28   
3         1     0           64  3869200167        2558        19   
4         1     0           64  3869200167        2760        25   
...     ...   ...          ...         ...         ...       ...   
9996      1     0           60  3869200267         701         5   
9997      1     0           59  3869200267         707        23   
9998      1     0           63  3869200267         706        15   
9999      1     0           49  3869200267         777        13   
10000     1     0           50  3869200267         776        22   

            TIME_IN_NS                      TIME_SRS  
0      344649004934875 1970-01-04 23:44:09.004934875  
1      344649004934875 1970-01-04 23:44:09.004934875  
2      34464900493

C:\Users\crnyl\AppData\Local\Temp\ipykernel_16296\2273251484.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_head['TIME_IN_NS'] = time_ns
C:\Users\crnyl\AppData\Local\Temp\ipykernel_16296\2273251484.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_head['TIME_SRS'] = time_new


In [7]:
# QUESTION 4
import datetime

max_ORBIT_CNT = data_head['ORBIT_CNT'].max()
print(max_ORBIT_CNT)

3869200267


In [8]:
#QUESTION 5

print(data_head[data_head['FPGA'] == 0].groupby(['TDC_CHANNEL']).count()['HEAD'].nlargest(3))
print(data_head[data_head['FPGA'] == 1].groupby(['TDC_CHANNEL']).count()['HEAD'].nlargest(3))


TDC_CHANNEL
139    599
63     496
64     480
Name: HEAD, dtype: int64
TDC_CHANNEL
139    261
2      250
1      237
Name: HEAD, dtype: int64


In [9]:
#QUESTION 6

non_empty = data_head.loc[data_head['TDC_MEAS']!=0]
print(non_empty.shape[0])

8845


In [10]:
#QUESTION 7
channel = data_head.loc[data_head['TDC_CHANNEL'] == 139]
measured = channel.loc[channel['TDC_MEAS'] != 0]
measured.nunique()['ORBIT_CNT']
print(channel.shape[0])

860


In [11]:
#QUESTION 8
fp_0 = data_head.loc[data_head['FPGA'] == 0]
new_series= pd.Series(fp_0['TDC_MEAS'].values,index= fp_0['TDC_CHANNEL'])

fp_1 = data_head.loc[data_head['FPGA'] == 1]
new_series_2= pd.Series(fp_1['TDC_MEAS'].values,index= fp_1['TDC_CHANNEL'])

print(new_series)
print(new_series_2)

TDC_CHANNEL
123    26
124    27
63     28
64     19
64     25
       ..
60      5
59     23
63     15
49     13
50     22
Length: 7016, dtype: int64
TDC_CHANNEL
7       4
6      18
139     0
8      14
5       4
       ..
139     0
29     12
30     16
24      6
88     12
Length: 2985, dtype: int64
